In [ ]:
import plotly.graph_objects as go
from scipy.signal import savgol_filter

In [ ]:
# QuantBook Analysis Tool
# For more information see [https://www.quantconnect.com/docs/v2/our-platform/research/getting-started]
qb = QuantBook()
start_date = datetime(2020, 1, 1)
end_date = datetime(2025, 1, 1)
spy = qb.add_equity("SPY", Resolution.DAILY).symbol
history = qb.history(qb.securities.keys(), start_date, end_date).droplevel(0)

history

In [ ]:
fig_raw = go.Figure()
fig_raw.add_trace(go.Scatter(x=history.index,  y=history.close))
fig_raw.show()

In [ ]:
close_prices = history.close.values # Numpy array of close prices
smooth_close_prices = savgol_filter(close_prices, window_length=63, polyorder=2)

fig = go.Figure()
fig.add_trace(go.Scatter(x=history.index, y=history.close)) # Raw vs Smooth
fig.add_trace(go.Scatter(x=history.index, y=smooth_close_prices))
fig.show()

In [ ]:
history['smoothed'] = smooth_close_prices
history

In [ ]:
history['positive_gradient'] = np.gradient(smooth_close_prices) > 0

history

In [ ]:
rsi = RelativeStrengthIndex(14)

# Get the indicator history of the indicator.
rsi_df = qb.indicator(rsi, spy, start_date, end_date)
df = history.loc[rsi_df.index]
t = rsi_df['current'].values > 70

In [ ]:
d = df['positive_gradient'].values == t
d.sum()/len(d)

In [ ]:
indicators = [
    # Trend Strength/Direction
    ('adx', AverageDirectionalIndex(20)),
    ('admxr', AverageDirectionalMovementIndexRating(20)),
    ('aroon', AroonOscillator(14, 14)),
    ('chop', ChoppinessIndex(14)),
    ('ikh', IchimokuKinkoHyo(7, 22, 44)),
    ('macd', MovingAverageConvergenceDivergence(12, 26, 9)),
    ('ppo', PercentagePriceOscillator(12, 26, 9)),  # MovingAverageType.Simple
    ('rsi', RelativeStrengthIndex(14)),
    ('rvi', RelativeVigorIndex(20, MovingAverageType.Simple)),
    ('tsi', TrueStrengthIndex(25, 13, 7)),
    ('vtx', Vortex(14)),
    ('mom', Momentum(14)),
    ('momp', MomentumPercent(14)),
    ('roc', RateOfChange(10)),
    ('rocp', RateOfChangePercent(10)),
    ('stc', SchaffTrendCycle(10, 23, 50)),
    ('ultosc', UltimateOscillator(7, 14, 28)),
    # Volatility and Range Expansion (Momentum Fuel)
    # ('atr', AverageTrueRange(14)),
    # ('natr', NormalizedAverageTrueRange(14)),
    ('bb', BollingerBands(20, 2)),
    ('kc', KeltnerChannels(20, 2)),
    ('sd', StandardDeviation(20)),
    ('rsv', RogersSatchellVolatility(20)),
    ('tr', TrueRange()),
    #  Moving Averages Suited for Trend Following
    ('ema20', ExponentialMovingAverage(20)),
    ('ema50', ExponentialMovingAverage(50)),
    ('ema100', ExponentialMovingAverage(100)),
    ('ema200', ExponentialMovingAverage(200)),
    ('dema20', DoubleExponentialMovingAverage(20)),
    ('dema50', DoubleExponentialMovingAverage(50)),
    ('dema100', DoubleExponentialMovingAverage(100)),
    ('dema200', DoubleExponentialMovingAverage(200)),
    ('tema20', TripleExponentialMovingAverage(20)),
    ('tema50', TripleExponentialMovingAverage(50)),
    ('tema100', TripleExponentialMovingAverage(100)),
    ('tema200', TripleExponentialMovingAverage(200)),
    ('hma20', HullMovingAverage(20)),
    ('hma50', HullMovingAverage(50)),
    ('hma100', HullMovingAverage(100)),
    ('hma200', HullMovingAverage(200)),
    # ('kama', KaufmanAdaptiveMovingAverage(20, 10, 20)),
    # ('frama', FractalAdaptiveMovingAverage(20, 198)),
    ('zlema20', ZeroLagExponentialMovingAverage(20)),
    ('zlema50', ZeroLagExponentialMovingAverage(50)),
    ('zlema100', ZeroLagExponentialMovingAverage(100)),
    ('zlema200', ZeroLagExponentialMovingAverage(200))
]

dfs = []
for name, ind in indicators:
    df = qb.indicator(ind, spy, start_date - timedelta(days=1000), end_date)
    df = df.loc[history.index]
    df.columns = [f'{name}_{x}' for x in df.columns]
    dfs.append(df)

indicator_df = pd.concat(dfs, axis=1)
indicator_df

In [ ]:
signals = [
    ('ADX > 25', indicator_df['adx_current'].values > 25),
    ('ADX > 20', indicator_df['adx_current'].values > 20),
    ('ADX > 15', indicator_df['adx_current'].values > 15),
    ('+DI > -DI', indicator_df['adx_positivedirectionalindex'].values > indicator_df['adx_negativedirectionalindex'].values),

    ('aroon > 0', indicator_df['aroon_current'].values > 0),
    ('aroon > 50', indicator_df['aroon_current'].values > 50),
    ('aroon > -50', indicator_df['aroon_current'].values > -50),

    ('chop > 38.2', indicator_df['chop_current'].values > 38.2),
    ('chop > 61.8', indicator_df['chop_current'].values > 61.8),

    ('SPY > senkouA', history['close'].values > indicator_df['ikh_senkoua'].values),
    ('SPY > senkouB', history['close'].values > indicator_df['ikh_senkoub'].values),
    ('tenkan > kijun', indicator_df['ikh_tenkan'].values > indicator_df['ikh_kijun'].values),
    ('SPY > chikou', history['close'].values > indicator_df['ikh_chikou'].values),

    ('macd > macds', indicator_df['macd_current'].values > indicator_df['macd_signal'].values),
    ('macdh rising', indicator_df['macd_histogram'].diff().values > 0),

    ('ppo > ppos', indicator_df['ppo_current'].values > indicator_df['ppo_signal'].values),
    ('ppoh rising', indicator_df['ppo_histogram'].diff().values > 0),

    ('rsi > 30', indicator_df['rsi_current'].values > 30),
    ('rsi > 50', indicator_df['rsi_current'].values > 50),
    ('rsi > 70', indicator_df['rsi_current'].values > 70),
    ('rsi rising', indicator_df['rsi_current'].diff().values > 0),

    ('rvi > rvis', indicator_df['rvi_current'].values > indicator_df['rvi_signal'].values),

    ('tsi > 0', indicator_df['tsi_current'].values > 0),
    ('tsi rising', indicator_df['tsi_current'].diff().values > 0),

    ('VI+ > VI-', indicator_df['vtx_plusvortex'].values > indicator_df['vtx_minusvortex'].values),

    ('mom > 0', indicator_df['mom_current'].values > 0),
    ('momp > 0', indicator_df['momp_current'].values > 0),

    ('roc > 0', indicator_df['roc_current'].values > 0),
    ('roc rising', indicator_df['roc_current'].diff().values > 0),
    ('rocp > 0', indicator_df['roc_current'].values > 0),
    ('rocp rising', indicator_df['roc_current'].diff().values > 0),

    ('stc > 25', indicator_df['stc_current'].values > 25),
    ('stc > 50', indicator_df['stc_current'].values > 50),
    ('stc > 75', indicator_df['stc_current'].values > 75),

    ('ultosc > 30', indicator_df['ultosc_current'].values > 30),

    ('SPY > BBU', history['close'].values > indicator_df['bb_upperband'].values),
    ('SPY > BBL', history['close'].values > indicator_df['bb_lowerband'].values),
    ('SPY > KCU', history['close'].values > indicator_df['kc_upperband'].values),
    ('vol rising', history['volume'].diff().values > 0), 
    ('SPY > KCL', history['close'].values > indicator_df['kc_lowerband'].values),
    
    ('SPY > EMA20', history['close'].values > indicator_df['ema20_current'].values),
    ('SPY > DEMA20', history['close'].values > indicator_df['dema20_current'].values),
    ('SPY > TEMA20', history['close'].values > indicator_df['tema20_current'].values),
    ('SPY > HMA20', history['close'].values > indicator_df['hma20_current'].values),
    ('SPY > ZLEMA20', history['close'].values > indicator_df['zlema20_current'].values),

    ('SPY > EMA50', history['close'].values > indicator_df['ema50_current'].values),
    ('SPY > DEMA50', history['close'].values > indicator_df['dema50_current'].values),
    ('SPY > TEMA50', history['close'].values > indicator_df['tema50_current'].values),
    ('SPY > HMA50', history['close'].values > indicator_df['hma50_current'].values),
    ('SPY > ZLEMA50', history['close'].values > indicator_df['zlema50_current'].values),

    ('SPY > EMA100', history['close'].values > indicator_df['ema100_current'].values),
    ('SPY > DEMA100', history['close'].values > indicator_df['dema100_current'].values),
    ('SPY > TEMA100', history['close'].values > indicator_df['tema100_current'].values),
    ('SPY > HMA100', history['close'].values > indicator_df['hma100_current'].values),
    ('SPY > ZLEMA100', history['close'].values > indicator_df['zlema100_current'].values),

    ('SPY > EMA200', history['close'].values < indicator_df['ema200_current'].values),
    ('SPY > DEMA200', history['close'].values < indicator_df['dema200_current'].values),
    ('SPY > TEMA200', history['close'].values < indicator_df['tema200_current'].values),
    ('SPY > HMA200', history['close'].values < indicator_df['hma200_current'].values),
    ('SPY > ZLEMA200', history['close'].values < indicator_df['zlema200_current'].values),

    ('EMA20 slope > 0', indicator_df['ema20_current'].diff().values > 0),
    ('DEMA20 slope > 0', indicator_df['dema20_current'].diff().values > 0),
    ('TEMA20 slope > 0', indicator_df['tema20_current'].diff().values > 0),
    ('HMA20 slope > 0', indicator_df['hma20_current'].diff().values > 0),
    ('ZLEMA20 slope > 0', indicator_df['zlema20_current'].diff().values > 0),

    ('EMA50 slope > 0', indicator_df['ema50_current'].diff().values > 0),
    ('DEMA50 slope > 0', indicator_df['dema50_current'].diff().values > 0),
    ('TEMA50 slope > 0', indicator_df['tema50_current'].diff().values > 0),
    ('HMA50 slope > 0', indicator_df['hma50_current'].diff().values > 0),
    ('ZLEMA50 slope > 0', indicator_df['zlema50_current'].diff().values > 0),

    ('EMA100 slope > 0', indicator_df['ema100_current'].diff().values > 0),
    ('DEMA100 slope > 0', indicator_df['dema100_current'].diff().values > 0),
    ('TEMA100 slope > 0', indicator_df['tema100_current'].diff().values > 0),
    ('HMA100 slope > 0', indicator_df['hma100_current'].diff().values > 0),
    ('ZLEMA100 slope > 0', indicator_df['zlema100_current'].diff().values > 0),

    ('EMA200 slope > 0', indicator_df['ema200_current'].diff().values > 0),
    ('DEMA200 slope > 0', indicator_df['dema200_current'].diff().values > 0),
    ('TEMA200 slope > 0', indicator_df['tema200_current'].diff().values > 0),
    ('HMA200 slope > 0', indicator_df['hma200_current'].diff().values > 0),
    ('ZLEMA200 slope > 0', indicator_df['zlema200_current'].diff().values > 0),

    ('EMA20 > EMA50', indicator_df['ema20_current'].values > indicator_df['ema50_current'].values),
    ('EMA20 > EMA100', indicator_df['ema20_current'].values > indicator_df['ema100_current'].values),
    ('EMA20 > EMA200', indicator_df['ema20_current'].values > indicator_df['ema200_current'].values),
    ('EMA50 > EMA100', indicator_df['ema50_current'].values > indicator_df['ema100_current'].values),
    ('EMA50 > EMA200', indicator_df['ema50_current'].values > indicator_df['ema200_current'].values),
    ('EMA100 > EMA200', indicator_df['ema100_current'].values > indicator_df['ema200_current'].values),

    ('DEMA20 > DEMA50', indicator_df['dema20_current'].values > indicator_df['dema50_current'].values),
    ('DEMA20 > DEMA100', indicator_df['dema20_current'].values > indicator_df['dema100_current'].values),
    ('DEMA20 > DEMA200', indicator_df['dema20_current'].values > indicator_df['dema200_current'].values),
    ('DEMA50 > DEMA100', indicator_df['dema50_current'].values > indicator_df['dema100_current'].values),
    ('DEMA50 > DEMA200', indicator_df['dema50_current'].values > indicator_df['dema200_current'].values),
    ('DEMA100 > DEMA200', indicator_df['dema100_current'].values > indicator_df['dema200_current'].values),

    ('TEMA20 > TEMA50', indicator_df['tema20_current'].values > indicator_df['tema50_current'].values),
    ('TEMA20 > TEMA100', indicator_df['tema20_current'].values > indicator_df['tema100_current'].values),
    ('TEMA20 > TEMA200', indicator_df['tema20_current'].values > indicator_df['tema200_current'].values),
    ('TEMA50 > TEMA100', indicator_df['tema50_current'].values > indicator_df['tema100_current'].values),
    ('TEMA50 > TEMA200', indicator_df['tema50_current'].values > indicator_df['tema200_current'].values),
    ('TEMA100 > TEMA200', indicator_df['tema100_current'].values > indicator_df['tema200_current'].values),

    ('HMA20 > HMA50', indicator_df['hma20_current'].values > indicator_df['hma50_current'].values),
    ('HMA20 > HMA100', indicator_df['hma20_current'].values > indicator_df['hma100_current'].values),
    ('HMA20 > HMA200', indicator_df['hma20_current'].values > indicator_df['hma200_current'].values),
    ('HMA50 > HMA100', indicator_df['hma50_current'].values > indicator_df['hma100_current'].values),
    ('HMA50 > HMA200', indicator_df['hma50_current'].values > indicator_df['hma200_current'].values),
    ('HMA100 > HMA200', indicator_df['hma100_current'].values > indicator_df['hma200_current'].values),

    ('ZLEMA20 > ZLEMA50', indicator_df['zlema20_current'].values > indicator_df['zlema50_current'].values),
    ('ZLEMA20 > ZLEMA100', indicator_df['zlema20_current'].values > indicator_df['zlema100_current'].values),
    ('ZLEMA20 > ZLEMA200', indicator_df['zlema20_current'].values > indicator_df['zlema200_current'].values),
    ('ZLEMA50 > ZLEMA100', indicator_df['zlema50_current'].values > indicator_df['zlema100_current'].values),
    ('ZLEMA50 > ZLEMA200', indicator_df['zlema50_current'].values > indicator_df['zlema200_current'].values),
    ('ZLEMA100 > ZLEMA200', indicator_df['zlema100_current'].values > indicator_df['zlema200_current'].values)
]


signals.extend([(title.replace(">", "<=").replace("rising", "falling"), np.bitwise_not(vector)) for title, vector in signals])
# len(signals)
signals



In [ ]:
signals = dict(signals)
truth = history['positive_gradient'].values

best_score = 0
best_combination = []
best_first_vector = None

for title, vector in signals.items():
    score = (vector == truth).sum() / len(vector)  # comparing ANDed vector to truth
    if score > best_score:
        best_score = score
        best_combination = [title]
        best_first_vector = vector

print(best_combination)

improvement = False
while True:
    for title, vector in signals.items():
        combo = vector & best_first_vector
        score = (combo == truth).sum() / len(vector)
        # print(score)
        if score > best_score:
            improvement = True
            best_score = score
            best_first_vector = combo
    if not improvement:
        break

print(best_score)

# combos = []
# for title1, vector1 in signals.items():
#     for title2, vector2 in signals.items():
#         combo = vector1 & vector2
#         score = (combo == truth).sum() / len(vector1)
#         combos.append((title1, title2, score))

# print(max(combos, key=lambda x: x[2]))

# fig = go.Figure()

# fig.add_trace(go.Scatter(x=history.index, y=history.close, name='SPY'))

# t = pd.Series(signals['SPY > EMA20'], index=indicator_df.index)
# t = t.astype(int).diff().dropna()
# t = t[t != 0]
# if len(t) > 0:
#     for i in range(1 if t.iloc[0] == -1 else 0, len(t) - 1, 2):
#         fig.add_vrect(
#             x0=t.index[i],
#             x1=t.index[i+1],
#             fillcolor='green',
#             layer='below',
#             line_width=0,
#             opacity=0.2
#         )

# fig.show()